In [ ]:
## Variables

set.seed(1985)
nMaquinas = 3
nOperaciones = 6
mProcesamiento = matrix(
  c(
    15, 12, NaN,
    7, 20, 10,
    NaN, 13, 20,
    15, NaN, 30, 
    25, 13, 14,
    15, 20, NaN
  ),
  nrow = nOperaciones,
  byrow = TRUE,
  dimnames = list(
    c("Ouno", "Odos", "Otres", "Ocuatro", "Ocinco", "Oseis"), 
    c("Muno", "Mdos", "Mtres")
  )
)
mAjuste = matrix(
  c(
    NaN, 5, 15, 10, 5, 35,
    NaN, NaN, 5, 15, 10, 45,
    NaN, NaN, NaN, 10, 20, 15,
    NaN, NaN, NaN, NaN, 7, 24,
    NaN, NaN, NaN, NaN, NaN, 13, 
    NaN, NaN, NaN,  NaN, NaN, NaN
  ),
  nrow = nOperaciones,
  byrow = TRUE,
  dimnames = list(
    c("Ouno", "Odos", "Otres", "Ocuatro", "Ocinco", "Oseis"), 
    c("Ouno", "Odos", "Otres", "Ocuatro", "Ocinco", "Oseis") 
  )
)

In [ ]:
# Funciones

getVectorRandom <- function(){
  v = rep(0, nOperaciones)
  for (oper in 1:nOperaciones){
    m <- sample(1:nMaquinas, 1)
    while (is.nan(mProcesamiento[oper, m])){
      m <- sample(1:nMaquinas, 1)
    }
    v[oper] <- m
  }
  return (v)
}

fObjetivo <- function(vecSol){
  res = 0
  for (maquina in 1:nMaquinas){
    nOp = 0
    totalM = 0
    i=0
    oper = 0
    operAnt = 0
    for (m in vecSol){
      oper = oper+1
      if(m == maquina){
        nOp = nOp+1
        totalM = totalM + mProcesamiento[oper, m]
        if(nOp>1){
          totalM = totalM + mAjuste[operAnt, oper]
        }
        operAnt = oper
      }
    }
    if (totalM > res){
      res = totalM
    }
  }
  return (res)
}

newTemp <- function(Temp){
  Tmp = Temp
  lr_aum = 1.02
  lr_dec = 0.98
  # 15% de probabilidad de aumentar la temperatura
  if(runif(1,0,1) < 0.15) Tmp = Tmp * lr_aum
  # 85% de probabilidad de disminuir la temperatura
  else                    Tmp = Tmp * lr_dec
  return (Tmp)
}

swapRandomVectorValues <- function(vec){

  newVector = vec
  swap = FALSE
  while(!swap){
    swap = TRUE
    # Seleccionar dos numeros aleatorios
    vals = sample.int(nOperaciones, 2)
    op1 = vals[1]
    op2 = vals[2]
    # Swapearlos si es posible, si no seguir
    if(is.nan(mProcesamiento[ op1, vec[op2] ])) swap = FALSE
    if(is.nan(mProcesamiento[ op2, vec[op1] ])) swap = FALSE
    if(swap){
      newVector[op1] = vec[op2]
      newVector[op2] = vec[op1]
    }
  }
  return (newVector)
}


In [ ]:
RecocidoS <- function(iteraciones, T_ini, T_minima, Show=FALSE){
  # Paso 0: Vector inicial; A cada operacion se le asigna la máquina que más le convenga
  #Elegimos un vector solucion de manera aleatoria
  vectorSolucion = getVectorRandom()
  mejorX = vectorSolucion
  Temp = T_ini
  if(Show) cat(mejorX, ": ", fObjetivo(mejorX), "\n")
  # for(i in 1: nrow(mProcesamiento)){ mejorX = c(mejorX, match(min(mProcesamiento[i, ], na.rm = TRUE), mProcesamiento[i, ])) }
  
  # Paso 4: Iterar hasta un maximo de iteraciones o alcanzar un valor minimo de T
  for(i in 1: iteraciones){
    if( Temp < T_minima) break

    # Paso 1: Generar vector propuesta
    
    # Propuesta 1: 
    # Probar valores random
    #vectorPropuesta = getVectorRandom()

    # Propuesta 2: 
    # Swapear dos operaciones random
    vectorPropuesta = swapRandomVectorValues(mejorX)

    if(Show) cat(vectorPropuesta, ": ", fObjetivo(vectorPropuesta), "\n")

    # Paso 2: Si f(xPropuesta) < f(xActual), definir xk+1 ← xActual. 
    # En caso contrario, si U(0, 1) < exp[(f(xActual) − f(xPropuesta))/T] definir xk+1 ← xPropuesta; si no xk+1 ← xActual.
    fObjPropuesta = fObjetivo(vectorPropuesta)
    fObjActual    = fObjetivo(mejorX)
    expValue = ( fObjActual - fObjPropuesta )/Temp  

    if( fObjPropuesta <  fObjActual || runif(1,0,1) < exp(expValue) ){
      mejorX = vectorPropuesta
    }
    
    # Paso 3: Enfriamento de la temperatura segun una funcion de enfriamiento
    Temp = newTemp(Temp)

  }
  cat("Mejor solucion", mejorX, ": ", fObjetivo(mejorX))
}

In [ ]:
RecocidoS(2000, 1000, 0, TRUE)

1 3 2 1 3 2 :  48 
1 3 2 1 3 2 :  48 
1 3 3 1 2 2 :  46 
2 3 3 1 1 2 :  67 
2 3 3 1 1 2 :  67 
2 3 3 1 1 2 :  67 
2 1 3 1 3 2 :  67 
1 2 3 1 3 2 :  85 
1 2 3 1 3 2 :  85 
2 2 3 1 3 1 :  54 
2 1 3 1 3 2 :  67 
2 1 3 1 3 2 :  67 
2 3 3 1 1 2 :  67 
2 1 3 3 1 2 :  67 
2 1 3 3 2 1 :  67 
2 1 3 3 2 1 :  67 
2 1 3 3 2 1 :  67 
2 1 3 3 2 1 :  67 
1 1 3 3 2 2 :  60 
1 1 3 3 2 2 :  60 
1 1 2 3 3 2 :  51 
2 1 2 3 3 1 :  67 
2 1 2 3 3 1 :  67 
1 2 2 3 3 1 :  65 
1 2 3 3 2 1 :  65 
1 2 3 3 2 1 :  65 
1 2 3 3 2 1 :  65 
1 2 3 3 2 1 :  65 
1 1 3 3 2 2 :  60 
1 1 3 3 2 2 :  60 
1 1 2 3 3 2 :  51 
1 3 2 3 1 2 :  55 
1 3 2 3 2 1 :  65 
1 3 2 3 2 1 :  65 
1 3 2 3 2 1 :  65 
1 3 2 3 2 1 :  65 
1 3 2 3 2 1 :  65 
1 3 2 3 2 1 :  65 
1 2 2 3 3 1 :  65 
1 2 2 3 3 1 :  65 
2 1 2 3 3 1 :  67 
2 1 2 3 3 1 :  67 
2 1 2 3 3 1 :  67 
2 1 3 3 2 1 :  67 
1 1 3 3 2 2 :  60 
1 3 3 1 2 2 :  46 
1 3 3 1 2 2 :  46 
1 3 2 1 3 2 :  48 
1 3 3 1 2 2 :  46 
1 3 3 1 2 2 :  46 
1 1 3 3 2 2 :  60 
1 1 3 3 2 2 :  60 
1 1 3 3 2 2 